In [1]:
%pip install --upgrade langsmith

import os

print("Langchain key to start tracing")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "pr-aching-poisoning-52"
os.environ["LANGCHAIN_API_KEY"] = "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"

Note: you may need to restart the kernel to use updated packages.
Langchain key to start tracing


In [2]:

import boto3


AWS_REGION = "us-east-1"

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION,
)

In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")


sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(client=bedrock_client,model_id="anthropic.claude-3-sonnet-20240229-v1:0")

In [5]:
llm.invoke("Hi").content

'Hello! How can I assist you today?'

In [6]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "For how many customers, company value is missing?"}).split('SQLQuery: ')[-1]
response

'SELECT COUNT(*) AS "Count" FROM "Customer" WHERE "Company" IS NULL;'

In [7]:
db.run(response)

'[(49,)]'

In [8]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result